In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import json 
import ast
import csv

In [2]:
movies=pd.read_csv('DataSets/movies_metadata.csv')

/home/abuzar/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
credits=pd.read_csv('DataSets/credits.csv')

In [4]:
credits=credits.drop(credits.columns[2],axis=1)

In [5]:
Data=movies.join(credits)

In [6]:
with open("crewlist.pkl", "rb") as f:
    FinalCrewList = pickle.load(f)
    f.close()

In [7]:
with open("castlist.pkl", "rb") as f:
    FinalCastList = pickle.load(f)
    f.close()

In [8]:
with open("movielist.pkl", "rb") as f:
    FinalMovieList = pickle.load(f)
    f.close()

In [9]:
FinalMovieList=FinalMovieList[0:1000]
FinalCrewList=FinalCrewList[0:1000]
FinalCastList=FinalCastList[0:1000]

In [10]:
def EctractMeaningful(List,Resultant,Size):
    index=0
    empty=True
    for i in range(0,Size,1):
        for j in range(0,len(List[i]),1):
            found = False
            temp=List[i][j]
            if temp!='':
                if(empty!=True):
                    for k in range(0,len(Resultant),1):
                        if(List[i][j]==Resultant[k]):
                            found=True
                    if found==False:
                        Resultant.insert(index, temp)
                        index+=1
                else:
                    empty=False
                    Resultant.insert(index, temp)
                    index+=1
    return Resultant

In [11]:
def DictToWords(List,Resultant,String,Size):
    feild=[]
    for i in range (0,len(List)):
        temp_list=[]
        for j in range (0,len(List[i])):
            temp_list.append(List[i][j][String])
        feild.append(temp_list)
    Resultant=EctractMeaningful(feild,Resultant,len(feild))
    return Resultant
Genres_=[]
Genres_=DictToWords(FinalMovieList,Genres_,'name',len(FinalMovieList))
jobs=[]
jobs=DictToWords(FinalCrewList,jobs,'job',len(FinalCrewList))

In [12]:
Genre_vector=np.zeros((len(FinalMovieList),len(jobs)), dtype=int)

In [13]:
def ToVect(List,Tags,String,Size):
    Resultant=np.zeros((len(FinalMovieList),len(Tags)), dtype=int)
    for i in range (0,Size):
        for j in range(0,len(List[i])):
            index=-1
            for k in range(0,len(Tags)):
                if List[i][j][String]==Tags[k]:
                    index=k
                    break
            if (index!=-1):
                Resultant[i][index]=1
    return Resultant


In [14]:
Genres_

['Animation',
 'Comedy',
 'Family',
 'Adventure',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'History',
 'Science Fiction',
 'Mystery',
 'War',
 'Foreign',
 'Music',
 'Documentary',
 'Western',
 'TV Movie']

In [16]:
Gen1 = open('Genres.pkl', 'wb')
pickle.dump(Genres_, Gen1)

In [10]:
Genre=ToVect(FinalMovieList,Genres_,'name',len(FinalMovieList))

In [11]:
GenresDF=pd.DataFrame(Genre,columns=Genres_)

In [12]:
Reqjobs=['Director']

In [13]:
temp=[]
Job_positions = {}
for i in jobs:
    Job_positions[i] = []
for i in range(0,len(FinalCrewList),1):
    for j in range(0,len(FinalCrewList[i]),1):
        
        index=-1
        for k in range(0,len(Reqjobs)):
            if FinalCrewList[i][j]['job']==Reqjobs[k]:
                index=k
        if (index!=-1):
            Job_positions[Reqjobs[index]].append(FinalCrewList[i][j]['name']) 

In [14]:
for i in range(0,len(Reqjobs)):
        Job_positions[Reqjobs[i]]=list(set(Job_positions[Reqjobs[i]]))

In [15]:
def ToVectUpdate(List,Tags,String,Type,ParentTag,Size):
    Resultant=np.zeros((len(FinalMovieList),len(Tags)), dtype=int)
    for i in range (0,Size):
        for j in range(0,len(List[i])):
            index=-1
            found=False
            if List[i][j][Type]==ParentTag:
                    found=True
            if found==True:
                for k in range(0,len(Tags)): 
                    if List[i][j][String]==Tags[k]:
                        index=k
                        break
            if (index!=-1):
                Resultant[i][index]=1
    return Resultant

In [16]:
Allcast= pd.DataFrame()
temp=[]
temp2=[]
#sum_=0
for i in range(0,len(Reqjobs)):
    Vectors=[]
    Vectors=ToVectUpdate(FinalCrewList,Job_positions[Reqjobs[i]],'name','job',Reqjobs[i],len(FinalCrewList))
    VectorsDf = pd.DataFrame(Vectors,columns=Job_positions[Reqjobs[i]])
    
print (VectorsDf.shape)


(1000, 796)


In [17]:
Actors=[]
Actors=DictToWords(FinalCastList,Actors,'name',len(FinalCastList))

In [18]:
actor_vector=ToVect(FinalCastList,Actors,'name',len(FinalCastList))

In [19]:
AllActors = pd.DataFrame(actor_vector,columns=Actors)

In [20]:
Final_DataFrame_vector=pd.DataFrame()

In [21]:
Final_DataFrame_vector=pd.concat([GenresDF,VectorsDf],axis=1)

In [22]:
Final_DataFrame_vector=pd.concat([Final_DataFrame_vector,AllActors],axis=1)

In [28]:
Final_DataFrame_vector=Final_DataFrame_vector.join(Data['id'])

In [ ]:
Final_DataFrame_vector.to_csv('Atributes_vectors.csv',index=False)